Used for testing and temporary code


In [25]:

string=time.ctime()
new=string.replace(" ",'_').replace(":","_")
print(new)

Tue_Apr__9_15_38_06_2024


In [ ]:
#setup etc
import carla
import numpy as np
import pygame
import cv2
import time
client = carla.Client('localhost', 2000)
client.set_timeout(120) #Enable longer wait time in case computer is slow

pygame 2.5.2 (SDL 2.28.3, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [11]:
world = client.get_world()
car_filter='*Ambulance*' 
vehicle_bp = world.get_blueprint_library().filter(car_filter)
blueprint_library= world.get_blueprint_library()

In [12]:
#Spawn a car at the spectator
spectator = world.get_spectator()
point=spectator.get_transform()
ego_vehicle=world.try_spawn_actor(vehicle_bp[0],point)


In [15]:
#add an instance segmentation camera
instance_segmentation_camera= blueprint_library.find('sensor.camera.instance_segmentation')
instance_segmentation_camera.set_attribute('sensor_tick','10')
camera_init_trans = carla.Transform(carla.Location(z=1.5)) 
camera = world.try_spawn_actor(instance_segmentation_camera, camera_init_trans, attach_to=ego_vehicle)

AttributeError: 'ServerSideSensor' object has no attribute 'set_attribute'

In [16]:
# Render object to keep and pass the PyGame surface
class RenderObject(object):
    def __init__(self, width, height):
        init_image = np.random.randint(0,255,(height,width,3),dtype='uint8')
        self.surface = pygame.surfarray.make_surface(init_image.swapaxes(0,1))

# Camera sensor callback, reshapes raw data from camera into 2D RGB and applies to PyGame surface
def pygame_callback(data, obj):
    img = np.reshape(np.copy(data.raw_data), (data.height, data.width, 4))
    img = img[:,:,:3]
    img = img[:, :, ::-1]
    obj.surface = pygame.surfarray.make_surface(img.swapaxes(0,1))

def camera_callback(image, data):
    capture = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data['image'] = capture
    cv2.imwrite(f"{image.frame}.png", capture)

In [18]:
camera.listen(lambda image: camera_callback(image, image.save_to_disk('/%.6d.jpg' % image.frame)))

: 